### Bonus Part 2: AlienNoFrameskip-v4

In [ ]:
print ('Submitted By')
print ('UBITname      = karanman')
print ('Person Number = 50290755')

### imports

In [ ]:
# for importing these packages you need to have the following packages installed on your system:
# gym
# cmake
# openmpi
# zlib
# stable-baselines
# ffmpeg

#importing packages
from stable_baselines.common.atari_wrappers import make_atari
from stable_baselines.common.vec_env import DummyVecEnv, VecVideoRecorder
from stable_baselines.deepq.policies import MlpPolicy, CnnPolicy
from stable_baselines import DQN

from stable_baselines.bench import Monitor
from stable_baselines.results_plotter import load_results, ts2xy

import numpy as np
import os
import matplotlib.pyplot as plt

### Callback

In [ ]:
best_mean_reward, n_steps = -np.inf, 0

def callback(_locals, _globals):
    """
    Callback called at each step (for DQN an others) or after n steps (see ACER or PPO2)
    :param _locals: (dict)
    :param _globals: (dict)
    """
    global n_steps, best_mean_reward
    # Print stats every 1000 calls
    if (n_steps + 1) % 1000 == 0:
        # Evaluate policy performance
        x, y = ts2xy(load_results(log_dir), 'timesteps')
        if len(x) > 0:
            mean_reward = np.mean(y[-100:])
            print(x[-1], 'timesteps')
            print("Best mean reward: {:.2f} - Last mean reward per episode: {:.2f}".format(best_mean_reward, mean_reward))

            # New best model, you could save the agent here
            if mean_reward > best_mean_reward:
                best_mean_reward = mean_reward
                # Example for saving best model
                print("Saving new best model")
                _locals['self'].save(log_dir + 'best_model.pkl')
    n_steps += 1
    return False

In [ ]:
# Create log dir
log_dir = "/tmp/atari/"
os.makedirs(log_dir, exist_ok=True)

In [ ]:
env_id = 'AlienNoFrameskip-v4'
video_folder = '2_ipynb_animation/'
video_length = 100

In [ ]:
# initialize environment
env = make_atari(env_id)
env = Monitor(env, log_dir, allow_early_resets=True)
env = DummyVecEnv([lambda: env])
env = VecVideoRecorder(env, video_folder,
                       record_video_trigger=lambda x: x == 0, video_length=video_length,
                       name_prefix="random-agent-{}".format(env_id))

### Uncomment the cells below to train instead of loading a trained model

In [ ]:
# model = DQN(CnnPolicy, env, verbose=1)

# model.learn(total_timesteps=10000, callback=callback)

### load trained model and play the game

In [ ]:
model = DQN.load("best_model")

In [ ]:
obs = env.reset()
for i in range(video_length + 1):
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    env.render()
    i+=1